In [15]:
import spacy
import json
import os
from nltk.corpus import words
import pandas as pd
import gensim
import gensim.corpora as corpora
import collections
import re, contractions, string
from sklearn.cluster import KMeans

In [2]:
# files = []
# subdir = tuple(os.walk("../imdb/imdb_movie_jsons/1990"))
# for file_name in tuple(subdir)[0][-1]:
#     with open(f"{subdir[0][0]}/{file_name}", 'r') as f:
#         files.append(json.load(f))

In [3]:
# ner = spacy.load("en_core_web_sm")
# test = "Speed was a surprise hit movie. While it cost about $30,000,000 to make, it made back more than four times the cost. So, some Hollywood executives (with a collective IQ similar to that of a gerbil) thought...'if we do a sequel, we must spend $120,000,000 on it because THAT is why the first movie was such a success'. Not surprisingly, the movie tanked and it somehow has made it to IMDB's infamous Bottom 100 list the 100 worst ranked major releases of all time.In the original, Keanu Reeves and Sandra Bullock work to prevent a bus from exploding and killing folks. Now, because this is an overbloated sequel, they've placed the story aboard a cruise ship...which is also out of control and will kill everyone unless our heroes save it....though this time Reeves refused to reprise his role. So, they cast the rather bland Jason Patric in the lead...who offered little improvement other than he was very good at using sign language in one small scene.How bad is this film? Well, not bad enough to be on the Bottom 100 list. So why is it on the list? After loving the first film, so many folks expected more of the same...and got nothing but a dull overblown action picture with little in the way to interest viewers....and I think THAT is why it made the list....because it disappointed so badly. Instead, I wish IMDB had a list for films with the worst ratio of cost to enjoyability of the film...in which case \"Speed 2\" would easily make the top 100.By the way, this sequel ended up spending nearly as much for ONE crash scene as the budget for all of \"Speed\"! And, after paying $11,500,000 just to obtain Sandra Bullock for this film, you can only assume that the film simply had nothing left to pay competent writers. Instead, I assume, they hired some penguins to write the script....some very, very dull and untalented penguins!"
# doc = ner(test)
# for ent in doc.ents:
#     print(f"{ent.text} {ent.label} {str (spacy.explain(ent.label_))}")

In [4]:

with open("../vibes.txt", 'r') as f:
    vibes = f.readlines()

In [5]:
nlp = spacy.load("en_core_web_lg")
corpus = []
for vibe in vibes:
    vibe_per_movie = nlp(vibe)
    vibe_collection = []
    for word in vibe_per_movie:
        if word.pos_ == "ADJ":
            vibe_collection.append(str(word.lemma_))
    corpus.append(" ".join(vibe_collection).strip())


In [6]:
eng_words = set(words.words())

def clean(text: str):
    
    # remove numbers
    text = re.sub(r"\d+", "", text)
    # expand contraction
    text = contractions.fix(text)
    # remove links
    text = re.sub(r"(ftp|http[s]?)://\S+", "", text)
    # remove punctuations
    sub_txt = r"[" + re.escape(string.punctuation) + r"]"
    text = re.sub(sub_txt, "", text)

    text = re.sub("[^a-zA-Z0-9 ]+", " ", text)

    # remove emails
    text = re.sub("[\w\.-]+@[\w\.-]+\.\w+", " ", text)
    
    text = " ".join([word for word in text.split() if word in eng_words and len(word) > 2])

     # remove multispaces
    text = re.sub('\s+', " ", text)

    # lemmatizer.lemmatize()
    return text.strip()

In [7]:
corpus = [clean(s) for s in corpus]

In [8]:
proc_data = [string.split() for string in corpus]
input_dict = corpora.Dictionary(proc_data)
input_corpus = [input_dict.doc2bow(token, allow_update=True) for token in proc_data]

In [9]:
model = gensim.models.TfidfModel(input_corpus, id2word=input_dict, normalize=True)
vector = model[input_corpus]

In [10]:
d = {}
for v in vector:
    for id, f in v:
        d[input_dict[id]] = f

In [11]:
df = pd.DataFrame(d.values(), index=d.keys(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)

In [12]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df)

                        TF-IDF
blinkered            6.202e-01
agitative            5.050e-01
freeze               4.278e-01
recast               4.126e-01
botcher              3.987e-01
collared             3.919e-01
incomprehensive      3.832e-01
chucky               3.821e-01
birdcall             3.801e-01
celluloid            3.730e-01
unendingness         3.712e-01
tern                 3.700e-01
untickled            3.680e-01
theek                3.669e-01
unhopeful            3.629e-01
rompy                3.619e-01
gud                  3.606e-01
meatless             3.602e-01
regret               3.544e-01
rightist             3.534e-01
snakeweed            3.506e-01
torture              3.499e-01
exothermic           3.474e-01
prefrontal           3.448e-01
vali                 3.406e-01
muss                 3.396e-01
hypersonic           3.365e-01
deva                 3.350e-01
climax               3.350e-01
terpsichorean        3.334e-01
monaural             3.253e-01
diversio

In [13]:
a = []
for word in corpus:
    a.extend(word.split())
# print(words)
a = dict(collections.Counter(a))
with open("../stopwords.json", "w") as stopwords:
    count_sw = [k for k,v in a.items() if v >= 1100]
    tfidf_sw = [k for k,v in d.items() if v <= 1.6 * 10**-3]
    json.dump(count_sw + tfidf_sw, stopwords)

In [16]:
a = {k: v for k, v in sorted(a.items(), key=lambda item: item[1], reverse=True)}

In [19]:
kmeans = KMeans(n_clusters=15, init="random", n_init=1250)
kmeans.fit(list(a.keys()), sample_weight=a.values())

ValueError: could not convert string to float: 'good'